In [1]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier


from Modules import Modules
from Baseline import Baseline
from FeatureEngineering import FeatureEngineering
from FeatureEngineering2 import FeatureEngineering2


%matplotlib inline

print("OK\n")

OK



In [2]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering = FeatureEngineering('SK_ID_CURR')
FeatureEngineering2 = FeatureEngineering2('SK_ID_CURR')

In [3]:
def ligthgbm_feature_selection(dataframe, index_cols, auc_limit=0.7):
        dataframe = dataframe.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '_', x))
        clf = LGBMClassifier(random_state=0)
        train_df = dataframe[dataframe['TARGET'].notnull()]
        train_df_X = train_df.drop('TARGET', axis=1)
        train_df_y = train_df['TARGET']
        train_columns = [col for col in train_df_X.columns if col not in index_cols]

        max_auc_score = 1
        best_cols = []
        while max_auc_score > auc_limit:
            train_columns = [col for col in train_columns if col not in best_cols]
            clf.fit(train_df_X[train_columns], train_df_y)
            feats_imp = pd.Series(clf.feature_importances_, index=train_columns)
            max_auc_score = roc_auc_score(train_df_y, clf.predict_proba(train_df_X[train_columns])[:, 1])
            best_cols = feats_imp[feats_imp > 0].index.tolist()

        dataframe.drop(train_columns, axis=1, inplace=True)
        return dataframe

In [4]:
"""
params_base = {
            'boosting_type': 'gbdt'
            ,'objective': 'binary'
            ,'metric': 'auc'
            ,'verbosity': -1
            ,'learning_rate': 0.05
            ,'n_estimators': 100000
            ,'random_state':123
            ,'bagging_freq': 1   
}
"""
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}



def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=100
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

In [5]:
path = './home-credit-default-risk/'

## ファイル読み込み

In [6]:
bureau = FeatureEngineering.bureau_and_balance(path)
bureau.head()

Memory usage of dataframe: 4114.69 MB
Memory usage after optimization: 1189.22 MB
Decreased by 71.1%


,BURO_SK_ID_CURR_MIN,BURO_SK_ID_CURR_MAX,BURO_SK_ID_CURR_SIZE,BURO_SK_ID_CURR_MEAN,BURO_SK_ID_CURR_VAR,BURO_SK_ID_CURR_SUM,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_SIZE,BURO_DAYS_CREDIT_MEAN,...,CLOSED_Last_status_3_MEAN_SIZE,CLOSED_Last_status_3_MEAN_MEAN,CLOSED_Last_status_3_MEAN_VAR,CLOSED_Last_status_3_MEAN_SUM,CLOSED_Last_status_4_MEAN_MIN,CLOSED_Last_status_4_MEAN_MAX,CLOSED_Last_status_4_MEAN_SIZE,CLOSED_Last_status_4_MEAN_MEAN,CLOSED_Last_status_4_MEAN_VAR,CLOSED_Last_status_4_MEAN_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,100001,100001,7,100001.0,0.0,700007,-1572,-49,7,-735.000,...,4.0,0.000000,0.000000,0.0,0.000000,0.000000,4.0,0.000000,0.000000,0.0
100002,100002,100002,8,100002.0,0.0,800016,-1437,-103,8,-874.000,...,6.0,0.000000,0.000000,0.0,0.000000,0.000000,6.0,0.000000,0.000000,0.0
100003,100003,100003,4,100003.0,0.0,400012,-2586,-606,4,-1401.000,...,3.0,1.400391,0.700195,0.0,1.400391,1.400391,3.0,1.400391,0.700195,0.0
100004,100004,100004,2,100004.0,0.0,200008,-1326,-408,2,-867.000,...,2.0,1.400391,0.700195,0.0,1.400391,1.400391,2.0,1.400391,0.700195,0.0
100005,100005,100005,3,100005.0,0.0,300015,-373,-62,3,-190.625,...,1.0,0.000000,0.700195,0.0,0.000000,0.000000,1.0,0.000000,0.700195,0.0


In [7]:
bureau = bureau.replace([np.inf, -np.inf], np.nan)
bureau = bureau.fillna(bureau.max() + (bureau.max() * 0.4))
bureau.head()

,BURO_SK_ID_CURR_MIN,BURO_SK_ID_CURR_MAX,BURO_SK_ID_CURR_SIZE,BURO_SK_ID_CURR_MEAN,BURO_SK_ID_CURR_VAR,BURO_SK_ID_CURR_SUM,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_SIZE,BURO_DAYS_CREDIT_MEAN,...,CLOSED_Last_status_3_MEAN_SIZE,CLOSED_Last_status_3_MEAN_MEAN,CLOSED_Last_status_3_MEAN_VAR,CLOSED_Last_status_3_MEAN_SUM,CLOSED_Last_status_4_MEAN_MIN,CLOSED_Last_status_4_MEAN_MAX,CLOSED_Last_status_4_MEAN_SIZE,CLOSED_Last_status_4_MEAN_MEAN,CLOSED_Last_status_4_MEAN_VAR,CLOSED_Last_status_4_MEAN_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,100001,100001,7,100001.0,0.0,700007,-1572,-49,7,-735.000,...,4.0,0.000000,0.000000,0.0,0.000000,0.000000,4.0,0.000000,0.000000,0.0
100002,100002,100002,8,100002.0,0.0,800016,-1437,-103,8,-874.000,...,6.0,0.000000,0.000000,0.0,0.000000,0.000000,6.0,0.000000,0.000000,0.0
100003,100003,100003,4,100003.0,0.0,400012,-2586,-606,4,-1401.000,...,3.0,1.400391,0.700195,0.0,1.400391,1.400391,3.0,1.400391,0.700195,0.0
100004,100004,100004,2,100004.0,0.0,200008,-1326,-408,2,-867.000,...,2.0,1.400391,0.700195,0.0,1.400391,1.400391,2.0,1.400391,0.700195,0.0
100005,100005,100005,3,100005.0,0.0,300015,-373,-62,3,-190.625,...,1.0,0.000000,0.700195,0.0,0.000000,0.000000,1.0,0.000000,0.700195,0.0


In [8]:
previous = FeatureEngineering.previous_application(path)
previous.head()

Memory usage of dataframe: 2434.34 MB
Memory usage after optimization: 818.55 MB
Decreased by 66.4%


,PREV_SK_ID_PREV_MIN,PREV_SK_ID_PREV_MAX,PREV_SK_ID_PREV_SIZE,PREV_SK_ID_PREV_MEAN,PREV_SK_ID_PREV_VAR,PREV_SK_ID_PREV_SUM,PREV_SK_ID_CURR_MIN,PREV_SK_ID_CURR_MAX,PREV_SK_ID_CURR_SIZE,PREV_SK_ID_CURR_MEAN,...,REFUSED_PRODUCT_COMBINATION_POS_other_with_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Card_X_Sell_MEAN,REFUSED_PRODUCT_COMBINATION_POS_mobile_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Card_Street_MEAN,REFUSED_PRODUCT_COMBINATION_POS_industry_with_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Cash_Street__low_MEAN,REFUSED_PRODUCT_COMBINATION_POS_industry_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Cash_Street__middle_MEAN,REFUSED_PRODUCT_COMBINATION_POS_others_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_nan_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1369693,1,1369693.00,2.367471e+12,1369693,100001,100001,1,100001.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100002,1038818,1038818,1,1038818.00,2.367471e+12,1038818,100002,100002,1,100002.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100003,1810518,2636178,3,2281150.25,1.804519e+11,6843451,100003,100003,3,100003.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100004,1564014,1564014,1,1564014.00,2.367471e+12,1564014,100004,100004,1,100004.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100005,1857999,2495675,2,2176837.00,2.033153e+11,4353674,100005,100005,2,100005.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391


In [9]:
previous = previous.replace([np.inf, -np.inf], np.nan)
previous = previous.fillna(previous.max() + (previous.max() * 0.4))
previous.head()

,PREV_SK_ID_PREV_MIN,PREV_SK_ID_PREV_MAX,PREV_SK_ID_PREV_SIZE,PREV_SK_ID_PREV_MEAN,PREV_SK_ID_PREV_VAR,PREV_SK_ID_PREV_SUM,PREV_SK_ID_CURR_MIN,PREV_SK_ID_CURR_MAX,PREV_SK_ID_CURR_SIZE,PREV_SK_ID_CURR_MEAN,...,REFUSED_PRODUCT_COMBINATION_POS_other_with_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Card_X_Sell_MEAN,REFUSED_PRODUCT_COMBINATION_POS_mobile_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Card_Street_MEAN,REFUSED_PRODUCT_COMBINATION_POS_industry_with_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Cash_Street__low_MEAN,REFUSED_PRODUCT_COMBINATION_POS_industry_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Cash_Street__middle_MEAN,REFUSED_PRODUCT_COMBINATION_POS_others_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_nan_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1369693,1,1369693.00,2.367471e+12,1369693,100001,100001,1,100001.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100002,1038818,1038818,1,1038818.00,2.367471e+12,1038818,100002,100002,1,100002.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100003,1810518,2636178,3,2281150.25,1.804519e+11,6843451,100003,100003,3,100003.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100004,1564014,1564014,1,1564014.00,2.367471e+12,1564014,100004,100004,1,100004.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100005,1857999,2495675,2,2176837.00,2.033153e+11,4353674,100005,100005,2,100005.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391


In [10]:
pos = FeatureEngineering.pos_cash(path)
pos.head()

Memory usage of dataframe: 147.95 MB
Memory usage after optimization: 46.96 MB
Decreased by 68.3%


,POS_SK_ID_PREV_MIN,POS_SK_ID_PREV_MAX,POS_SK_ID_PREV_SIZE,POS_SK_ID_PREV_MEAN,POS_SK_ID_PREV_VAR,POS_SK_ID_PREV_SUM,POS_SK_ID_CURR_MIN,POS_SK_ID_CURR_MAX,POS_SK_ID_CURR_SIZE,POS_SK_ID_CURR_MEAN,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1851984,9,1584044.50,6.461239e+10,14256401,100001,100001,9,100001.0,...,0.777832,0.222168,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,9
100002,1038818,1038818,19,1038818.00,0.000000e+00,19737542,100002,100002,19,100002.0,...,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,19
100003,1810518,2636178,28,2297665.25,1.086316e+11,64334628,100003,100003,28,100003.0,...,0.928711,0.071411,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,28
100004,1564014,1564014,4,1564014.00,0.000000e+00,6256056,100004,100004,4,100004.0,...,0.750000,0.250000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,4
100005,2495675,2495675,11,2495675.00,0.000000e+00,27452425,100005,100005,11,100005.0,...,0.818359,0.090881,0.090881,0.0,0.0,0.0,0.0,0.0,0.0,11


In [11]:
pos = pos.replace([np.inf, -np.inf], np.nan)
pos = pos.fillna(pos.max() + (pos.max() * 0.4))
pos.head()

,POS_SK_ID_PREV_MIN,POS_SK_ID_PREV_MAX,POS_SK_ID_PREV_SIZE,POS_SK_ID_PREV_MEAN,POS_SK_ID_PREV_VAR,POS_SK_ID_PREV_SUM,POS_SK_ID_CURR_MIN,POS_SK_ID_CURR_MAX,POS_SK_ID_CURR_SIZE,POS_SK_ID_CURR_MEAN,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1851984,9,1584044.50,6.461239e+10,14256401,100001,100001,9,100001.0,...,0.777832,0.222168,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,9
100002,1038818,1038818,19,1038818.00,0.000000e+00,19737542,100002,100002,19,100002.0,...,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,19
100003,1810518,2636178,28,2297665.25,1.086316e+11,64334628,100003,100003,28,100003.0,...,0.928711,0.071411,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,28
100004,1564014,1564014,4,1564014.00,0.000000e+00,6256056,100004,100004,4,100004.0,...,0.750000,0.250000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,4
100005,2495675,2495675,11,2495675.00,0.000000e+00,27452425,100005,100005,11,100005.0,...,0.818359,0.090881,0.090881,0.0,0.0,0.0,0.0,0.0,0.0,11


In [12]:
installments = FeatureEngineering.installments_payments(path)
installments.head()

Memory usage of dataframe: 199.49 MB
Memory usage after optimization: 73.84 MB
Decreased by 63.0%


,INS_SK_ID_PREV_MIN,INS_SK_ID_PREV_MAX,INS_SK_ID_PREV_SIZE,INS_SK_ID_PREV_MEAN,INS_SK_ID_PREV_VAR,INS_SK_ID_PREV_SUM,INS_SK_ID_CURR_MIN,INS_SK_ID_CURR_MAX,INS_SK_ID_CURR_SIZE,INS_SK_ID_CURR_MEAN,...,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_MEAN,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_VAR,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_SUM,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MIN,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MAX,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_SIZE,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MEAN,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_VAR,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_SUM,INSTAL_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1851984,7,1576389.125,6.645846e+10,11034724,100001,100001,7,100001.0,...,0.0,0.0,0.0,1.0,1.0,7,1.0,0.0,7.0,7
100002,1038818,1038818,19,1038818.000,0.000000e+00,19737542,100002,100002,19,100002.0,...,0.0,0.0,0.0,1.0,1.0,19,1.0,0.0,19.0,19
100003,1810518,2636178,25,2290070.250,1.027131e+11,57251754,100003,100003,25,100003.0,...,0.0,0.0,0.0,1.0,1.0,25,1.0,0.0,25.0,25
100004,1564014,1564014,3,1564014.000,0.000000e+00,4692042,100004,100004,3,100004.0,...,0.0,0.0,0.0,1.0,1.0,3,1.0,0.0,3.0,3
100005,2495675,2495675,9,2495675.000,0.000000e+00,22461075,100005,100005,9,100005.0,...,0.0,0.0,0.0,1.0,1.0,9,1.0,0.0,9.0,9


In [13]:
installments = installments.replace([np.inf, -np.inf], np.nan)
installments = installments.fillna(installments.max() + (installments.max() * 0.4))
installments.head()

,INS_SK_ID_PREV_MIN,INS_SK_ID_PREV_MAX,INS_SK_ID_PREV_SIZE,INS_SK_ID_PREV_MEAN,INS_SK_ID_PREV_VAR,INS_SK_ID_PREV_SUM,INS_SK_ID_CURR_MIN,INS_SK_ID_CURR_MAX,INS_SK_ID_CURR_SIZE,INS_SK_ID_CURR_MEAN,...,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_MEAN,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_VAR,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_SUM,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MIN,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MAX,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_SIZE,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MEAN,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_VAR,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_SUM,INSTAL_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1851984,7,1576389.125,6.645846e+10,11034724,100001,100001,7,100001.0,...,0.0,0.0,0.0,1.0,1.0,7,1.0,0.0,7.0,7
100002,1038818,1038818,19,1038818.000,0.000000e+00,19737542,100002,100002,19,100002.0,...,0.0,0.0,0.0,1.0,1.0,19,1.0,0.0,19.0,19
100003,1810518,2636178,25,2290070.250,1.027131e+11,57251754,100003,100003,25,100003.0,...,0.0,0.0,0.0,1.0,1.0,25,1.0,0.0,25.0,25
100004,1564014,1564014,3,1564014.000,0.000000e+00,4692042,100004,100004,3,100004.0,...,0.0,0.0,0.0,1.0,1.0,3,1.0,0.0,3.0,3
100005,2495675,2495675,9,2495675.000,0.000000e+00,22461075,100005,100005,9,100005.0,...,0.0,0.0,0.0,1.0,1.0,9,1.0,0.0,9.0,9


In [14]:
credit = FeatureEngineering.credit_card_balance(path)
credit.head()

Memory usage of dataframe: 173.03 MB
Memory usage after optimization: 66.37 MB
Decreased by 61.6%


,CARD_SK_ID_PREV_MIN,CARD_SK_ID_PREV_MAX,CARD_SK_ID_PREV_SIZE,CARD_SK_ID_PREV_MEAN,CARD_SK_ID_PREV_VAR,CARD_SK_ID_PREV_SUM,CARD_SK_ID_CURR_MIN,CARD_SK_ID_CURR_MAX,CARD_SK_ID_CURR_SIZE,CARD_SK_ID_CURR_MEAN,...,CARD_card_AMT_DRAWINGS_CURRENT__AMT_DRAWINGS_POS_CURRENT_VAR,CARD_card_AMT_DRAWINGS_CURRENT__AMT_DRAWINGS_POS_CURRENT_SUM,CARD_NAME_CONTRACT_STATUS_Active_MEAN,CARD_NAME_CONTRACT_STATUS_Completed_MEAN,CARD_NAME_CONTRACT_STATUS_Demand_MEAN,CARD_NAME_CONTRACT_STATUS_Signed_MEAN,CARD_NAME_CONTRACT_STATUS_Sent_proposal_MEAN,CARD_NAME_CONTRACT_STATUS_Refused_MEAN,CARD_NAME_CONTRACT_STATUS_Approved_MEAN,CARD_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100006,1489396,1489396,6,1489396.0,0.0,8936376,100006,100006,6,100006.0,...,8.505000e+11,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,6
100011,1843384,1843384,74,1843384.0,0.0,136410416,100011,100011,74,100011.0,...,4.378378e+08,180000.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,74
100013,2038692,2038692,96,2038692.0,0.0,195714432,100013,100013,96,100013.0,...,8.249688e+08,571500.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,96
100021,2594025,2594025,17,2594025.0,0.0,44098425,100021,100021,17,100021.0,...,8.505000e+11,0.0,0.411865,0.588379,0.0,0.0,0.0,0.0,0.0,17
100023,1499902,1499902,8,1499902.0,0.0,11999216,100023,100023,8,100023.0,...,8.505000e+11,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,8


In [15]:
credit = credit.replace([np.inf, -np.inf], np.nan)
credit = credit.fillna(credit.max() + (credit.max() * 0.4))
credit.head()

,CARD_SK_ID_PREV_MIN,CARD_SK_ID_PREV_MAX,CARD_SK_ID_PREV_SIZE,CARD_SK_ID_PREV_MEAN,CARD_SK_ID_PREV_VAR,CARD_SK_ID_PREV_SUM,CARD_SK_ID_CURR_MIN,CARD_SK_ID_CURR_MAX,CARD_SK_ID_CURR_SIZE,CARD_SK_ID_CURR_MEAN,...,CARD_card_AMT_DRAWINGS_CURRENT__AMT_DRAWINGS_POS_CURRENT_VAR,CARD_card_AMT_DRAWINGS_CURRENT__AMT_DRAWINGS_POS_CURRENT_SUM,CARD_NAME_CONTRACT_STATUS_Active_MEAN,CARD_NAME_CONTRACT_STATUS_Completed_MEAN,CARD_NAME_CONTRACT_STATUS_Demand_MEAN,CARD_NAME_CONTRACT_STATUS_Signed_MEAN,CARD_NAME_CONTRACT_STATUS_Sent_proposal_MEAN,CARD_NAME_CONTRACT_STATUS_Refused_MEAN,CARD_NAME_CONTRACT_STATUS_Approved_MEAN,CARD_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100006,1489396,1489396,6,1489396.0,0.0,8936376,100006,100006,6,100006.0,...,8.505000e+11,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,6
100011,1843384,1843384,74,1843384.0,0.0,136410416,100011,100011,74,100011.0,...,4.378378e+08,180000.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,74
100013,2038692,2038692,96,2038692.0,0.0,195714432,100013,100013,96,100013.0,...,8.249688e+08,571500.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,96
100021,2594025,2594025,17,2594025.0,0.0,44098425,100021,100021,17,100021.0,...,8.505000e+11,0.0,0.411865,0.588379,0.0,0.0,0.0,0.0,0.0,17
100023,1499902,1499902,8,1499902.0,0.0,11999216,100023,100023,8,100023.0,...,8.505000e+11,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,8


In [16]:
df = FeatureEngineering.application_train_test(path, 'application_train.csv')
df.head()

Memory usage of dataframe: 404.30 MB
Memory usage after optimization: 169.87 MB
Decreased by 58.0%


,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,app_AMT_INCOME_TOTAL_div_CNT_CHILDREN,app_most_popular_AMT_GOODS_PRICE,app_popular_AMT_GOODS_PRICE,app_OWN_CAR_AGE_div_DAYS_BIRTH,app_OWN_CAR_AGE_div_DAYS_EMPLOYED,app_DAYS_LAST_PHONE_CHANGE_div_DAYS_BIRTH,app_DAYS_LAST_PHONE_CHANGE_div_DAYS_EMPLOYED,app_DAYS_EMPLOYED_PERC,app_DAYS_EMPLOYED_div_DAYS_BIRTH,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,202500.0,0,0,NaN,NaN,0.119873,1.780273,8824.0,0.067322,0.0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,270000.0,0,0,NaN,NaN,0.049377,0.696777,15576.0,0.070862,0.0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,67500.0,0,0,-0.001365,-0.11554,0.042786,3.623047,18816.0,0.011810,0.0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,135000.0,0,0,NaN,NaN,0.032471,0.203003,15968.0,0.159912,0.0
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,121500.0,0,0,NaN,NaN,0.055481,0.364014,16896.0,0.152466,0.0


### データフレームを結合

In [17]:
### データフレームをマージ
df = pd.merge(df, bureau, on='SK_ID_CURR', how='left')
df = pd.merge(df, installments, on='SK_ID_CURR', how='left')
df = pd.merge(df, previous, on='SK_ID_CURR', how='left')
df = pd.merge(df, credit, on='SK_ID_CURR', how='left')
df = pd.merge(df, pos, on='SK_ID_CURR', how='left')
df.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,1.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,19.0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0.928711,0.071411,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,28.0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0.750000,0.250000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,4.0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0.856934,0.095215,0.000000,0.0,0.047607,0.0,0.0,0.0,0.0,21.0
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,0.939453,0.045441,0.015152,0.0,0.000000,0.0,0.0,0.0,0.0,66.0


### 欠損値補完

In [18]:
### 処理対象の列名を配列化
replace_columns = list(df.select_dtypes(include='number').columns)
replace_columns.remove('SK_ID_CURR')
replace_columns.remove('TARGET')

In [19]:
for column in replace_columns:
    df[column].fillna(df[column].max() + (df[column].max() * 0.4))

df.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,1.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,19.0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0.928711,0.071411,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,28.0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0.750000,0.250000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,4.0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0.856934,0.095215,0.000000,0.0,0.047607,0.0,0.0,0.0,0.0,21.0
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,0.939453,0.045441,0.015152,0.0,0.000000,0.0,0.0,0.0,0.0,66.0


### 変数選択

In [20]:
### 変数選択
df = FeatureEngineering2.data_post_processing(df)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 3396 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 3279 features are remained after removing non-informative features
1142 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 2137 features are remained after removing features not interesting for LightGBM classifier
---=> 90 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 2227 features


,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0,0,0,0,0,0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0,0,0,0,0,0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0,0,0,0,0,0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,1,0,0,1,0,1,0,1,0,1
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0,0,0,0,0,0


### 標準化・対数変換

In [21]:
### 処理対象の列名を配列化
encoding_columns = list(df.select_dtypes(include='number').columns)
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('TARGET')

In [22]:
### indexを設定
df = df.set_index('SK_ID_CURR')
df.head()

,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,0.018799,...,0,0,0,0,0,0,0,0,0,0
100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,...,0,0,0,0,0,0,0,0,0,0
100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,0.010033,...,0,0,0,0,0,0,0,0,0,0
100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,0.008018,...,1,0,0,1,0,1,0,1,0,1
100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,0.028656,...,0,0,0,0,0,0,0,0,0,0


In [23]:
### 標準化
scaler = StandardScaler()

for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    scaler.fit(df[num_cols])
    df[num_cols] = scaler.transform(df[num_cols])
        
df.head()

,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1.0,-1.393267,-0.717262,-0.665217,-0.575136,0.303184,-0.454491,-0.184969,-0.483578,-0.152222,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
100003,0.0,0.717738,-0.717262,1.503268,-0.575136,0.928775,1.770415,0.561525,1.643092,-1.249023,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
100004,0.0,-1.393267,1.394191,-0.665217,-0.575136,-0.947998,-1.135825,-1.403366,-1.073637,-0.782227,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
100006,0.0,0.717738,-0.717262,-0.665217,-0.575136,-0.322407,-0.690088,0.153458,-0.631093,-0.927246,...,1.665660,-0.162544,-0.504112,3.218467,-0.504033,3.218467,-0.504138,3.219143,-0.329824,2.047603
100007,0.0,-1.393267,-0.717262,-0.665217,-0.575136,-0.447525,-0.187568,-0.377395,-0.041034,0.555664,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376


In [24]:
### indexを削除
df = df.reset_index()
df.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
0,100002,1.0,-1.393267,-0.717262,-0.665217,-0.575136,0.303184,-0.454491,-0.184969,-0.483578,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
1,100003,0.0,0.717738,-0.717262,1.503268,-0.575136,0.928775,1.770415,0.561525,1.643092,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
2,100004,0.0,-1.393267,1.394191,-0.665217,-0.575136,-0.947998,-1.135825,-1.403366,-1.073637,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
3,100006,0.0,0.717738,-0.717262,-0.665217,-0.575136,-0.322407,-0.690088,0.153458,-0.631093,...,1.665660,-0.162544,-0.504112,3.218467,-0.504033,3.218467,-0.504138,3.219143,-0.329824,2.047603
4,100007,0.0,-1.393267,-0.717262,-0.665217,-0.575136,-0.447525,-0.187568,-0.377395,-0.041034,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376


In [25]:
### Yao-Jhonson変換
### 学習データに基づいてYao-Johnson変換を定義
#for column in encoding_columns:
#    pt = PowerTransformer(method = 'yeo-johnson')
#    pt.fit(df[column].copy())

    ### 変換後のデータで各列を置換
#    df[column] = pt.transform(df[column])

### train/testを分割

In [26]:
app_train = df[df['TARGET'].notnull()]
app_test = df[df['TARGET'].isnull()]

print(app_test.shape)
app_test.head()

(48744, 2227)


,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
307505,100001,NaN,0.717738,-0.717262,-0.665217,-0.575136,-0.322407,-0.047587,-0.465973,-0.213135,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
307506,100005,NaN,-1.393267,-0.717262,-0.665217,-0.575136,-0.656056,-0.915649,-0.682529,-0.950708,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
307507,100013,NaN,-1.393267,1.394191,-0.665217,-0.575136,0.303184,0.189387,2.874617,0.278581,...,1.665660,-0.162544,1.983688,-0.310707,1.983998,-0.310707,1.983584,-0.310642,-0.329824,2.047603
307508,100028,NaN,0.717738,-0.717262,-0.665217,2.201191,1.345836,2.476585,1.465625,2.860089,...,1.665660,-0.162544,1.983688,-0.310707,1.983998,-0.310707,1.983584,-0.310642,-0.329824,2.047603
307509,100038,NaN,-1.393267,1.394191,1.503268,0.813027,0.094654,0.094652,0.315035,0.266288,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376


In [27]:
del df
del bureau
del installments
del previous
del credit
del pos

gc.collect()

0

In [28]:
app_test.drop('TARGET', axis=1, inplace=True)
print(app_test.shape)
app_test.head()

(48744, 2226)


,SK_ID_CURR,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
307505,100001,0.717738,-0.717262,-0.665217,-0.575136,-0.322407,-0.047587,-0.465973,-0.213135,-0.148926,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
307506,100005,-1.393267,-0.717262,-0.665217,-0.575136,-0.656056,-0.915649,-0.682529,-0.950708,1.069336,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
307507,100013,-1.393267,1.394191,-0.665217,-0.575136,0.303184,0.189387,2.874617,0.278581,-0.130371,...,1.665660,-0.162544,1.983688,-0.310707,1.983998,-0.310707,1.983584,-0.310642,-0.329824,2.047603
307508,100028,0.717738,-0.717262,-0.665217,2.201191,1.345836,2.476585,1.465625,2.860089,0.393555,...,1.665660,-0.162544,1.983688,-0.310707,1.983998,-0.310707,1.983584,-0.310642,-0.329824,2.047603
307509,100038,-1.393267,1.394191,1.503268,0.813027,0.094654,0.094652,0.315035,0.266288,-0.782227,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376


In [29]:
#app_train = app_train.replace([np.inf, -np.inf], np.nan)
#app_train = app_train.fillna(app_train.max() + (app_train.max() * 0.4))
app_train.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
0,100002,1.0,-1.393267,-0.717262,-0.665217,-0.575136,0.303184,-0.454491,-0.184969,-0.483578,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
1,100003,0.0,0.717738,-0.717262,1.503268,-0.575136,0.928775,1.770415,0.561525,1.643092,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
2,100004,0.0,-1.393267,1.394191,-0.665217,-0.575136,-0.947998,-1.135825,-1.403366,-1.073637,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
3,100006,0.0,0.717738,-0.717262,-0.665217,-0.575136,-0.322407,-0.690088,0.153458,-0.631093,...,1.665660,-0.162544,-0.504112,3.218467,-0.504033,3.218467,-0.504138,3.219143,-0.329824,2.047603
4,100007,0.0,-1.393267,-0.717262,-0.665217,-0.575136,-0.447525,-0.187568,-0.377395,-0.041034,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376


In [30]:
#app_test = app_test.replace([np.inf, -np.inf], np.nan)
#app_test = app_test.fillna(app_test.max() + (app_test.max() * 0.4))
app_test.head()

,SK_ID_CURR,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
307505,100001,0.717738,-0.717262,-0.665217,-0.575136,-0.322407,-0.047587,-0.465973,-0.213135,-0.148926,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
307506,100005,-1.393267,-0.717262,-0.665217,-0.575136,-0.656056,-0.915649,-0.682529,-0.950708,1.069336,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
307507,100013,-1.393267,1.394191,-0.665217,-0.575136,0.303184,0.189387,2.874617,0.278581,-0.130371,...,1.665660,-0.162544,1.983688,-0.310707,1.983998,-0.310707,1.983584,-0.310642,-0.329824,2.047603
307508,100028,0.717738,-0.717262,-0.665217,2.201191,1.345836,2.476585,1.465625,2.860089,0.393555,...,1.665660,-0.162544,1.983688,-0.310707,1.983998,-0.310707,1.983584,-0.310642,-0.329824,2.047603
307509,100038,-1.393267,1.394191,1.503268,0.813027,0.094654,0.094652,0.315035,0.266288,-0.782227,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376


## CSVファイルを出力

In [31]:
df_train = app_train.copy()
df_train.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
0,100002,1.0,-1.393267,-0.717262,-0.665217,-0.575136,0.303184,-0.454491,-0.184969,-0.483578,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
1,100003,0.0,0.717738,-0.717262,1.503268,-0.575136,0.928775,1.770415,0.561525,1.643092,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
2,100004,0.0,-1.393267,1.394191,-0.665217,-0.575136,-0.947998,-1.135825,-1.403366,-1.073637,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
3,100006,0.0,0.717738,-0.717262,-0.665217,-0.575136,-0.322407,-0.690088,0.153458,-0.631093,...,1.665660,-0.162544,-0.504112,3.218467,-0.504033,3.218467,-0.504138,3.219143,-0.329824,2.047603
4,100007,0.0,-1.393267,-0.717262,-0.665217,-0.575136,-0.447525,-0.187568,-0.377395,-0.041034,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376


In [32]:
df_train.to_csv(
    path_or_buf="./home-credit-default-risk/exports/df_train.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [33]:
df_test = app_test.copy()
df_test.head()

,SK_ID_CURR,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
307505,100001,0.717738,-0.717262,-0.665217,-0.575136,-0.322407,-0.047587,-0.465973,-0.213135,-0.148926,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
307506,100005,-1.393267,-0.717262,-0.665217,-0.575136,-0.656056,-0.915649,-0.682529,-0.950708,1.069336,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
307507,100013,-1.393267,1.394191,-0.665217,-0.575136,0.303184,0.189387,2.874617,0.278581,-0.130371,...,1.665660,-0.162544,1.983688,-0.310707,1.983998,-0.310707,1.983584,-0.310642,-0.329824,2.047603
307508,100028,0.717738,-0.717262,-0.665217,2.201191,1.345836,2.476585,1.465625,2.860089,0.393555,...,1.665660,-0.162544,1.983688,-0.310707,1.983998,-0.310707,1.983584,-0.310642,-0.329824,2.047603
307509,100038,-1.393267,1.394191,1.503268,0.813027,0.094654,0.094652,0.315035,0.266288,-0.782227,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376


In [34]:
df_test.to_csv(
    path_or_buf="./home-credit-default-risk/exports/df_test.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [35]:
del app_train
del app_test
gc.collect()

0

## CSVファイルを読み込み

## 前処理

In [36]:
for col in df_train.columns:
    df_train[col] = df_train[col].replace('\s', '', regex=True)
    df_train[col] = df_train[col].replace(':', '', regex=True)
    df_train[col] = df_train[col].replace('/', '', regex=True)
    df_train[col] = df_train[col].replace(r'\(', r'', regex=True)
    df_train[col] = df_train[col].replace(r'\)', r'', regex=True)
    df_train[col] = df_train[col].replace(r'\+', r'', regex=True)
    df_train[col] = df_train[col].replace(r'\,', r'', regex=True)
    print(col)

SK_ID_CURR
TARGET
CODE_GENDER
FLAG_OWN_CAR
FLAG_OWN_REALTY
CNT_CHILDREN
AMT_INCOME_TOTAL
AMT_CREDIT
AMT_ANNUITY
AMT_GOODS_PRICE
REGION_POPULATION_RELATIVE
DAYS_BIRTH
DAYS_EMPLOYED
DAYS_REGISTRATION
DAYS_ID_PUBLISH
OWN_CAR_AGE
FLAG_EMP_PHONE
FLAG_WORK_PHONE
FLAG_PHONE
FLAG_EMAIL
CNT_FAM_MEMBERS
REGION_RATING_CLIENT
REGION_RATING_CLIENT_W_CITY
HOUR_APPR_PROCESS_START
REG_REGION_NOT_WORK_REGION
LIVE_REGION_NOT_WORK_REGION
REG_CITY_NOT_LIVE_CITY
REG_CITY_NOT_WORK_CITY
LIVE_CITY_NOT_WORK_CITY
EXT_SOURCE_1
EXT_SOURCE_2
EXT_SOURCE_3
APARTMENTS_AVG
BASEMENTAREA_AVG
YEARS_BEGINEXPLUATATION_AVG
YEARS_BUILD_AVG
COMMONAREA_AVG
ELEVATORS_AVG
ENTRANCES_AVG
FLOORSMAX_AVG
FLOORSMIN_AVG
LANDAREA_AVG
LIVINGAPARTMENTS_AVG
LIVINGAREA_AVG
NONLIVINGAPARTMENTS_AVG
NONLIVINGAREA_AVG
APARTMENTS_MODE
BASEMENTAREA_MODE
YEARS_BEGINEXPLUATATION_MODE
YEARS_BUILD_MODE
COMMONAREA_MODE
ELEVATORS_MODE
ENTRANCES_MODE
FLOORSMAX_MODE
FLOORSMIN_MODE
LANDAREA_MODE
LIVINGAPARTMENTS_MODE
LIVINGAREA_MODE
NONLIVINGAPARTMENTS_MO

BURO_bureau_DAYS_CREDIT__DAYS_CREDIT_ENDDATE_SUM
BURO_bureau_DAYS_CREDIT__DAYS_ENDDATE_FACT_MIN
BURO_bureau_DAYS_CREDIT__DAYS_ENDDATE_FACT_MAX
BURO_bureau_DAYS_CREDIT__DAYS_ENDDATE_FACT_MEAN
BURO_bureau_DAYS_CREDIT__DAYS_ENDDATE_FACT_VAR
BURO_bureau_DAYS_CREDIT__DAYS_ENDDATE_FACT_SUM
BURO_bureau_DAYS_CREDIT_ENDDATE__DAYS_ENDDATE_FACT_MIN
BURO_bureau_DAYS_CREDIT_ENDDATE__DAYS_ENDDATE_FACT_MAX
BURO_bureau_DAYS_CREDIT_ENDDATE__DAYS_ENDDATE_FACT_MEAN
BURO_bureau_DAYS_CREDIT_ENDDATE__DAYS_ENDDATE_FACT_VAR
BURO_bureau_DAYS_CREDIT_ENDDATE__DAYS_ENDDATE_FACT_SUM
BURO_bureau_DAYS_CREDIT_UPDATE__DAYS_CREDIT_ENDDATE_MIN
BURO_bureau_DAYS_CREDIT_UPDATE__DAYS_CREDIT_ENDDATE_MAX
BURO_bureau_DAYS_CREDIT_UPDATE__DAYS_CREDIT_ENDDATE_MEAN
BURO_bureau_DAYS_CREDIT_UPDATE__DAYS_CREDIT_ENDDATE_VAR
BURO_bureau_DAYS_CREDIT_UPDATE__DAYS_CREDIT_ENDDATE_SUM
BURO_CREDIT_ACTIVE_Closed_MEAN
BURO_CREDIT_ACTIVE_Active_MEAN
BURO_CREDIT_ACTIVE_Sold_MEAN
BURO_CREDIT_CURRENCY_currency_1_MEAN
BURO_CREDIT_CURRENCY_currency_

ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_VAR
ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_DEBT_SUM
ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_LIMIT_MIN
ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_LIMIT_MAX
ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_LIMIT_MEAN
ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_LIMIT_VAR
ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_LIMIT_SUM
ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_OVERDUE_MIN
ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_OVERDUE_MAX
ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_OVERDUE_MEAN
ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_OVERDUE_VAR
ACTIVE_bureau_AMT_CREDIT_SUM__AMT_CREDIT_SUM_OVERDUE_SUM
ACTIVE_bureau_DAYS_CREDIT__CREDIT_DAY_OVERDUE_MIN
ACTIVE_bureau_DAYS_CREDIT__CREDIT_DAY_OVERDUE_MAX
ACTIVE_bureau_DAYS_CREDIT__CREDIT_DAY_OVERDUE_MEAN
ACTIVE_bureau_DAYS_CREDIT__CREDIT_DAY_OVERDUE_VAR
ACTIVE_bureau_DAYS_CREDIT__CREDIT_DAY_OVERDUE_SUM
ACTIVE_bureau_DAYS_CREDIT__DAYS_CREDIT_ENDDATE_MIN
ACTIVE_bureau_DAYS_CREDIT__D

CLOSED_bureau_DAYS_CREDIT__DAYS_CREDIT_ENDDATE_MEAN
CLOSED_bureau_DAYS_CREDIT__DAYS_CREDIT_ENDDATE_VAR
CLOSED_bureau_DAYS_CREDIT__DAYS_CREDIT_ENDDATE_SUM
CLOSED_bureau_DAYS_CREDIT__DAYS_ENDDATE_FACT_MIN
CLOSED_bureau_DAYS_CREDIT__DAYS_ENDDATE_FACT_MAX
CLOSED_bureau_DAYS_CREDIT__DAYS_ENDDATE_FACT_MEAN
CLOSED_bureau_DAYS_CREDIT__DAYS_ENDDATE_FACT_VAR
CLOSED_bureau_DAYS_CREDIT__DAYS_ENDDATE_FACT_SUM
CLOSED_bureau_DAYS_CREDIT_ENDDATE__DAYS_ENDDATE_FACT_MIN
CLOSED_bureau_DAYS_CREDIT_ENDDATE__DAYS_ENDDATE_FACT_MAX
CLOSED_bureau_DAYS_CREDIT_ENDDATE__DAYS_ENDDATE_FACT_MEAN
CLOSED_bureau_DAYS_CREDIT_ENDDATE__DAYS_ENDDATE_FACT_VAR
CLOSED_bureau_DAYS_CREDIT_ENDDATE__DAYS_ENDDATE_FACT_SUM
CLOSED_bureau_DAYS_CREDIT_UPDATE__DAYS_CREDIT_ENDDATE_MIN
CLOSED_bureau_DAYS_CREDIT_UPDATE__DAYS_CREDIT_ENDDATE_MAX
CLOSED_bureau_DAYS_CREDIT_UPDATE__DAYS_CREDIT_ENDDATE_MEAN
CLOSED_bureau_DAYS_CREDIT_UPDATE__DAYS_CREDIT_ENDDATE_VAR
CLOSED_bureau_DAYS_CREDIT_UPDATE__DAYS_CREDIT_ENDDATE_SUM
CLOSED_CREDIT_CURRENCY_

PREV_AMT_APPLICATION_VAR
PREV_AMT_APPLICATION_SUM
PREV_AMT_CREDIT_MIN
PREV_AMT_CREDIT_MAX
PREV_AMT_CREDIT_MEAN
PREV_AMT_CREDIT_VAR
PREV_AMT_CREDIT_SUM
PREV_AMT_DOWN_PAYMENT_MIN
PREV_AMT_DOWN_PAYMENT_MAX
PREV_AMT_DOWN_PAYMENT_MEAN
PREV_AMT_DOWN_PAYMENT_VAR
PREV_AMT_DOWN_PAYMENT_SUM
PREV_AMT_GOODS_PRICE_MIN
PREV_AMT_GOODS_PRICE_MAX
PREV_AMT_GOODS_PRICE_MEAN
PREV_AMT_GOODS_PRICE_VAR
PREV_AMT_GOODS_PRICE_SUM
PREV_HOUR_APPR_PROCESS_START_MIN
PREV_HOUR_APPR_PROCESS_START_MAX
PREV_HOUR_APPR_PROCESS_START_MEAN
PREV_HOUR_APPR_PROCESS_START_VAR
PREV_HOUR_APPR_PROCESS_START_SUM
PREV_NFLAG_LAST_APPL_IN_DAY_MEAN
PREV_NFLAG_LAST_APPL_IN_DAY_VAR
PREV_NFLAG_LAST_APPL_IN_DAY_SUM
PREV_RATE_DOWN_PAYMENT_MIN
PREV_RATE_DOWN_PAYMENT_MAX
PREV_RATE_DOWN_PAYMENT_MEAN
PREV_RATE_DOWN_PAYMENT_VAR
PREV_RATE_DOWN_PAYMENT_SUM
PREV_RATE_INTEREST_PRIMARY_MIN
PREV_RATE_INTEREST_PRIMARY_MAX
PREV_RATE_INTEREST_PRIMARY_MEAN
PREV_RATE_INTEREST_PRIMARY_SUM
PREV_RATE_INTEREST_PRIVILEGED_MIN
PREV_RATE_INTEREST_PRIVILEGED_MEAN

PREV_NAME_SELLER_INDUSTRY_Tourism_MEAN
PREV_NAME_YIELD_GROUP_middle_MEAN
PREV_NAME_YIELD_GROUP_low_action_MEAN
PREV_NAME_YIELD_GROUP_high_MEAN
PREV_NAME_YIELD_GROUP_low_normal_MEAN
PREV_NAME_YIELD_GROUP_XNA_MEAN
PREV_PRODUCT_COMBINATION_POS_mobile_with_interest_MEAN
PREV_PRODUCT_COMBINATION_Cash_X_Sell__low_MEAN
PREV_PRODUCT_COMBINATION_Cash_X_Sell__high_MEAN
PREV_PRODUCT_COMBINATION_Cash_X_Sell__middle_MEAN
PREV_PRODUCT_COMBINATION_Cash_Street__high_MEAN
PREV_PRODUCT_COMBINATION_Cash_MEAN
PREV_PRODUCT_COMBINATION_POS_household_without_interest_MEAN
PREV_PRODUCT_COMBINATION_POS_household_with_interest_MEAN
PREV_PRODUCT_COMBINATION_POS_other_with_interest_MEAN
PREV_PRODUCT_COMBINATION_Card_X_Sell_MEAN
PREV_PRODUCT_COMBINATION_POS_mobile_without_interest_MEAN
PREV_PRODUCT_COMBINATION_Card_Street_MEAN
PREV_PRODUCT_COMBINATION_POS_industry_with_interest_MEAN
PREV_PRODUCT_COMBINATION_Cash_Street__low_MEAN
PREV_PRODUCT_COMBINATION_POS_industry_without_interest_MEAN
PREV_PRODUCT_COMBINATION_C

APPROVED_CHANNEL_TYPE_Regional___Local_MEAN
APPROVED_CHANNEL_TYPE_AP___Cash_loan__MEAN
APPROVED_CHANNEL_TYPE_Channel_of_corporate_sales_MEAN
APPROVED_NAME_SELLER_INDUSTRY_Connectivity_MEAN
APPROVED_NAME_SELLER_INDUSTRY_XNA_MEAN
APPROVED_NAME_SELLER_INDUSTRY_Consumer_electronics_MEAN
APPROVED_NAME_SELLER_INDUSTRY_Industry_MEAN
APPROVED_NAME_SELLER_INDUSTRY_Clothing_MEAN
APPROVED_NAME_SELLER_INDUSTRY_Furniture_MEAN
APPROVED_NAME_SELLER_INDUSTRY_Construction_MEAN
APPROVED_NAME_SELLER_INDUSTRY_Jewelry_MEAN
APPROVED_NAME_SELLER_INDUSTRY_Auto_technology_MEAN
APPROVED_NAME_YIELD_GROUP_middle_MEAN
APPROVED_NAME_YIELD_GROUP_low_action_MEAN
APPROVED_NAME_YIELD_GROUP_high_MEAN
APPROVED_NAME_YIELD_GROUP_low_normal_MEAN
APPROVED_PRODUCT_COMBINATION_POS_mobile_with_interest_MEAN
APPROVED_PRODUCT_COMBINATION_Cash_X_Sell__low_MEAN
APPROVED_PRODUCT_COMBINATION_Cash_X_Sell__high_MEAN
APPROVED_PRODUCT_COMBINATION_Cash_X_Sell__middle_MEAN
APPROVED_PRODUCT_COMBINATION_Cash_Street__high_MEAN
APPROVED_PRODUC

CARD_AMT_DRAWINGS_ATM_CURRENT_MIN
CARD_AMT_DRAWINGS_ATM_CURRENT_MAX
CARD_AMT_DRAWINGS_ATM_CURRENT_MEAN
CARD_AMT_DRAWINGS_ATM_CURRENT_VAR
CARD_AMT_DRAWINGS_ATM_CURRENT_SUM
CARD_AMT_DRAWINGS_CURRENT_MIN
CARD_AMT_DRAWINGS_CURRENT_MAX
CARD_AMT_DRAWINGS_CURRENT_MEAN
CARD_AMT_DRAWINGS_CURRENT_VAR
CARD_AMT_DRAWINGS_CURRENT_SUM
CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN
CARD_AMT_DRAWINGS_OTHER_CURRENT_MAX
CARD_AMT_DRAWINGS_OTHER_CURRENT_MEAN
CARD_AMT_DRAWINGS_OTHER_CURRENT_VAR
CARD_AMT_DRAWINGS_OTHER_CURRENT_SUM
CARD_AMT_DRAWINGS_POS_CURRENT_MIN
CARD_AMT_DRAWINGS_POS_CURRENT_MAX
CARD_AMT_DRAWINGS_POS_CURRENT_MEAN
CARD_AMT_DRAWINGS_POS_CURRENT_VAR
CARD_AMT_DRAWINGS_POS_CURRENT_SUM
CARD_AMT_INST_MIN_REGULARITY_MIN
CARD_AMT_INST_MIN_REGULARITY_MAX
CARD_AMT_INST_MIN_REGULARITY_MEAN
CARD_AMT_INST_MIN_REGULARITY_VAR
CARD_AMT_INST_MIN_REGULARITY_SUM
CARD_AMT_PAYMENT_CURRENT_MIN
CARD_AMT_PAYMENT_CURRENT_MAX
CARD_AMT_PAYMENT_CURRENT_MEAN
CARD_AMT_PAYMENT_CURRENT_VAR
CARD_AMT_PAYMENT_CURRENT_SUM
CARD_AMT_PAYME

ACTIVE_CNT_CREDIT_PROLONG_MAX_high_risk
ACTIVE_CNT_CREDIT_PROLONG_MAX_low_risk
ACTIVE_CREDIT_CURRENCY_currency_1_MEAN_high_risk
ACTIVE_CREDIT_CURRENCY_currency_1_MEAN_low_risk
ACTIVE_CREDIT_TYPE_Loan_for_working_capital_replenishment_MEAN_high_risk
ACTIVE_CREDIT_TYPE_Loan_for_working_capital_replenishment_MEAN_low_risk
ACTIVE_DPD_3_cnt_MIN_SUM_high_risk
ACTIVE_DPD_3_cnt_MIN_SUM_low_risk
ACTIVE_First_status_X_MEAN_SUM_high_risk
ACTIVE_First_status_X_MEAN_SUM_low_risk
ACTIVE_First_status_0_MEAN_SUM_high_risk
ACTIVE_First_status_0_MEAN_SUM_low_risk
ACTIVE_Last_status_C_MEAN_SUM_high_risk
ACTIVE_Last_status_C_MEAN_SUM_low_risk
ACTIVE_Last_status_X_MEAN_SUM_high_risk
ACTIVE_Last_status_X_MEAN_SUM_low_risk
ACTIVE_Last_status_0_MEAN_SUM_high_risk
ACTIVE_Last_status_0_MEAN_SUM_low_risk
CLOSED_Last_status_1_MEAN_SUM_high_risk
CLOSED_Last_status_1_MEAN_SUM_low_risk
INS_NUM_INSTALMENT_VERSION_MIN_high_risk
INS_NUM_INSTALMENT_VERSION_MIN_low_risk
PREV_NFLAG_INSURED_ON_APPROVAL_SUM_high_risk
PREV_N

In [37]:
df_train = df_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df_train.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
0,100002,1.0,-1.393267,-0.717262,-0.665217,-0.575136,0.303184,-0.454491,-0.184969,-0.483578,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
1,100003,0.0,0.717738,-0.717262,1.503268,-0.575136,0.928775,1.770415,0.561525,1.643092,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
2,100004,0.0,-1.393267,1.394191,-0.665217,-0.575136,-0.947998,-1.135825,-1.403366,-1.073637,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
3,100006,0.0,0.717738,-0.717262,-0.665217,-0.575136,-0.322407,-0.690088,0.153458,-0.631093,...,1.665660,-0.162544,-0.504112,3.218467,-0.504033,3.218467,-0.504138,3.219143,-0.329824,2.047603
4,100007,0.0,-1.393267,-0.717262,-0.665217,-0.575136,-0.447525,-0.187568,-0.377395,-0.041034,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376


In [38]:
for col in df_test.columns:
    df_test[col] = df_test[col].replace('\s', '', regex=True)
    df_test[col] = df_test[col].replace(':', '', regex=True)
    df_test[col] = df_test[col].replace('/', '', regex=True)
    df_test[col] = df_test[col].replace(r'\(', r'', regex=True)
    df_test[col] = df_test[col].replace(r'\)', r'', regex=True)
    df_test[col] = df_test[col].replace(r'\+', r'', regex=True)
    df_test[col] = df_test[col].replace(r'\,', r'', regex=True)
    print(col)

SK_ID_CURR
CODE_GENDER
FLAG_OWN_CAR
FLAG_OWN_REALTY
CNT_CHILDREN
AMT_INCOME_TOTAL
AMT_CREDIT
AMT_ANNUITY
AMT_GOODS_PRICE
REGION_POPULATION_RELATIVE
DAYS_BIRTH
DAYS_EMPLOYED
DAYS_REGISTRATION
DAYS_ID_PUBLISH
OWN_CAR_AGE
FLAG_EMP_PHONE
FLAG_WORK_PHONE
FLAG_PHONE
FLAG_EMAIL
CNT_FAM_MEMBERS
REGION_RATING_CLIENT
REGION_RATING_CLIENT_W_CITY
HOUR_APPR_PROCESS_START
REG_REGION_NOT_WORK_REGION
LIVE_REGION_NOT_WORK_REGION
REG_CITY_NOT_LIVE_CITY
REG_CITY_NOT_WORK_CITY
LIVE_CITY_NOT_WORK_CITY
EXT_SOURCE_1
EXT_SOURCE_2
EXT_SOURCE_3
APARTMENTS_AVG
BASEMENTAREA_AVG
YEARS_BEGINEXPLUATATION_AVG
YEARS_BUILD_AVG
COMMONAREA_AVG
ELEVATORS_AVG
ENTRANCES_AVG
FLOORSMAX_AVG
FLOORSMIN_AVG
LANDAREA_AVG
LIVINGAPARTMENTS_AVG
LIVINGAREA_AVG
NONLIVINGAPARTMENTS_AVG
NONLIVINGAREA_AVG
APARTMENTS_MODE
BASEMENTAREA_MODE
YEARS_BEGINEXPLUATATION_MODE
YEARS_BUILD_MODE
COMMONAREA_MODE
ELEVATORS_MODE
ENTRANCES_MODE
FLOORSMAX_MODE
FLOORSMIN_MODE
LANDAREA_MODE
LIVINGAPARTMENTS_MODE
LIVINGAREA_MODE
NONLIVINGAPARTMENTS_MODE
NONL

BURO_Month_MAX_MEAN
BURO_Month_MAX_VAR
BURO_Month_MAX_SUM
BURO_Month_SIZE_MIN
BURO_Month_SIZE_MAX
BURO_Month_SIZE_MEAN
BURO_Month_SIZE_VAR
BURO_Month_SIZE_SUM
BURO_When_closed_MIN_MIN
BURO_When_closed_MIN_MAX
BURO_When_closed_MIN_MEAN
BURO_When_closed_MIN_VAR
BURO_When_closed_MIN_SUM
BURO_When_closed_MAX_MIN
BURO_When_closed_MAX_MAX
BURO_When_closed_MAX_MEAN
BURO_When_closed_MAX_VAR
BURO_When_closed_MAX_SUM
BURO_When_closed_SIZE_SUM
BURO_Month_closed_to_end_MIN_MIN
BURO_Month_closed_to_end_MIN_MAX
BURO_Month_closed_to_end_MIN_MEAN
BURO_Month_closed_to_end_MIN_VAR
BURO_Month_closed_to_end_MIN_SUM
BURO_Month_closed_to_end_MAX_MIN
BURO_Month_closed_to_end_MAX_MAX
BURO_Month_closed_to_end_MAX_MEAN
BURO_Month_closed_to_end_MAX_VAR
BURO_Month_closed_to_end_MAX_SUM
BURO_DPD_0_cnt_MIN_MIN
BURO_DPD_0_cnt_MIN_MAX
BURO_DPD_0_cnt_MIN_MEAN
BURO_DPD_0_cnt_MIN_VAR
BURO_DPD_0_cnt_MIN_SUM
BURO_DPD_0_cnt_MAX_MIN
BURO_DPD_0_cnt_MAX_MAX
BURO_DPD_0_cnt_MAX_MEAN
BURO_DPD_0_cnt_MAX_VAR
BURO_DPD_0_cnt_MAX_SUM

ACTIVE_STATUS_C_MEAN_VAR
ACTIVE_STATUS_C_MEAN_SUM
ACTIVE_STATUS_0_MEAN_MIN
ACTIVE_STATUS_0_MEAN_MAX
ACTIVE_STATUS_0_MEAN_MEAN
ACTIVE_STATUS_0_MEAN_VAR
ACTIVE_STATUS_0_MEAN_SUM
ACTIVE_STATUS_X_MEAN_MIN
ACTIVE_STATUS_X_MEAN_MAX
ACTIVE_STATUS_X_MEAN_MEAN
ACTIVE_STATUS_X_MEAN_VAR
ACTIVE_STATUS_X_MEAN_SUM
ACTIVE_STATUS_1_MEAN_MIN
ACTIVE_STATUS_1_MEAN_MAX
ACTIVE_STATUS_1_MEAN_MEAN
ACTIVE_STATUS_1_MEAN_VAR
ACTIVE_STATUS_1_MEAN_SUM
ACTIVE_STATUS_2_MEAN_MIN
ACTIVE_STATUS_2_MEAN_MAX
ACTIVE_STATUS_2_MEAN_MEAN
ACTIVE_STATUS_2_MEAN_SUM
ACTIVE_STATUS_3_MEAN_SUM
ACTIVE_First_status_X_MEAN_MEAN
ACTIVE_First_status_X_MEAN_VAR
ACTIVE_First_status_X_MEAN_SUM
ACTIVE_First_status_0_MEAN_MIN
ACTIVE_First_status_0_MEAN_MAX
ACTIVE_First_status_0_MEAN_MEAN
ACTIVE_First_status_0_MEAN_VAR
ACTIVE_First_status_0_MEAN_SUM
ACTIVE_Last_status_C_MEAN_MAX
ACTIVE_Last_status_C_MEAN_MEAN
ACTIVE_Last_status_C_MEAN_VAR
ACTIVE_Last_status_C_MEAN_SUM
ACTIVE_Last_status_X_MEAN_MAX
ACTIVE_Last_status_X_MEAN_MEAN
ACTIVE_Last_st

PREV_SK_ID_PREV_SIZE
PREV_SK_ID_PREV_MEAN
PREV_SK_ID_PREV_VAR
PREV_SK_ID_PREV_SUM
PREV_SK_ID_CURR_MIN
PREV_SK_ID_CURR_MAX
PREV_SK_ID_CURR_SIZE
PREV_SK_ID_CURR_SUM
PREV_AMT_ANNUITY_MIN
PREV_AMT_ANNUITY_MAX
PREV_AMT_ANNUITY_SIZE
PREV_AMT_ANNUITY_MEAN
PREV_AMT_ANNUITY_VAR
PREV_AMT_ANNUITY_SUM
PREV_AMT_APPLICATION_MIN
PREV_AMT_APPLICATION_MAX
PREV_AMT_APPLICATION_SIZE
PREV_AMT_APPLICATION_MEAN
PREV_AMT_APPLICATION_VAR
PREV_AMT_APPLICATION_SUM
PREV_AMT_CREDIT_MIN
PREV_AMT_CREDIT_MAX
PREV_AMT_CREDIT_MEAN
PREV_AMT_CREDIT_VAR
PREV_AMT_CREDIT_SUM
PREV_AMT_DOWN_PAYMENT_MIN
PREV_AMT_DOWN_PAYMENT_MAX
PREV_AMT_DOWN_PAYMENT_MEAN
PREV_AMT_DOWN_PAYMENT_VAR
PREV_AMT_DOWN_PAYMENT_SUM
PREV_AMT_GOODS_PRICE_MIN
PREV_AMT_GOODS_PRICE_MAX
PREV_AMT_GOODS_PRICE_MEAN
PREV_AMT_GOODS_PRICE_VAR
PREV_AMT_GOODS_PRICE_SUM
PREV_HOUR_APPR_PROCESS_START_MIN
PREV_HOUR_APPR_PROCESS_START_MAX
PREV_HOUR_APPR_PROCESS_START_MEAN
PREV_HOUR_APPR_PROCESS_START_VAR
PREV_HOUR_APPR_PROCESS_START_SUM
PREV_NFLAG_LAST_APPL_IN_DAY_MEAN


APPROVED_NAME_CASH_LOAN_PURPOSE_XAP_MEAN
APPROVED_NAME_CASH_LOAN_PURPOSE_XNA_MEAN
APPROVED_NAME_CASH_LOAN_PURPOSE_Repairs_MEAN
APPROVED_NAME_CASH_LOAN_PURPOSE_Everyday_expenses_MEAN
APPROVED_NAME_CASH_LOAN_PURPOSE_Car_repairs_MEAN
APPROVED_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex_MEAN
APPROVED_NAME_CASH_LOAN_PURPOSE_Other_MEAN
APPROVED_NAME_CASH_LOAN_PURPOSE_Journey_MEAN
APPROVED_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment_MEAN
APPROVED_NAME_CASH_LOAN_PURPOSE_Medicine_MEAN
APPROVED_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans_MEAN
APPROVED_NAME_CASH_LOAN_PURPOSE_Urgent_needs_MEAN
APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car_MEAN
APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_MEAN
APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN
APPROVED_NAME_CASH_LOAN_PURPOSE_Business_development_MEAN
APPROVED_NAME_PAYMENT_TYPE_Cash_through_the_bank_MEAN
APPROVED_NAME_PAYMENT_TYPE_XNA_MEAN
APPROVED_NAME_PAYMENT_TYPE_Non_cash_from_your_account_MEAN
APPROVED_NAME_PAYMENT

REFUSED_CODE_REJECT_REASON_SYSTEM_MEAN
REFUSED_NAME_TYPE_SUITE_nan_MEAN
REFUSED_NAME_TYPE_SUITE_Unaccompanied_MEAN
REFUSED_NAME_TYPE_SUITE_Spouse_partner_MEAN
REFUSED_NAME_TYPE_SUITE_Family_MEAN
REFUSED_NAME_TYPE_SUITE_Children_MEAN
REFUSED_NAME_TYPE_SUITE_Other_B_MEAN
REFUSED_NAME_CLIENT_TYPE_Repeater_MEAN
REFUSED_NAME_CLIENT_TYPE_New_MEAN
REFUSED_NAME_CLIENT_TYPE_Refreshed_MEAN
REFUSED_NAME_GOODS_CATEGORY_Mobile_MEAN
REFUSED_NAME_GOODS_CATEGORY_XNA_MEAN
REFUSED_NAME_GOODS_CATEGORY_Consumer_Electronics_MEAN
REFUSED_NAME_GOODS_CATEGORY_Auto_Accessories_MEAN
REFUSED_NAME_GOODS_CATEGORY_Photo___Cinema_Equipment_MEAN
REFUSED_NAME_GOODS_CATEGORY_Computers_MEAN
REFUSED_NAME_GOODS_CATEGORY_Audio_Video_MEAN
REFUSED_NAME_GOODS_CATEGORY_Furniture_MEAN
REFUSED_NAME_GOODS_CATEGORY_Vehicles_MEAN
REFUSED_NAME_PORTFOLIO_POS_MEAN
REFUSED_NAME_PORTFOLIO_Cash_MEAN
REFUSED_NAME_PORTFOLIO_XNA_MEAN
REFUSED_NAME_PORTFOLIO_Cards_MEAN
REFUSED_NAME_PRODUCT_TYPE_XNA_MEAN
REFUSED_NAME_PRODUCT_TYPE_x_sell_MEAN
R

PREV_prev_missing_MAX_high_risk
PREV_prev_missing_MAX_low_risk
APPROVED_prev_missing_MAX_high_risk
APPROVED_prev_missing_MAX_low_risk
APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_MEAN_high_risk
APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car_MEAN_low_risk
APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_high_risk
APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_low_risk
APPROVED_NAME_CASH_LOAN_PURPOSE_Business_development_MEAN_high_risk
APPROVED_NAME_CASH_LOAN_PURPOSE_Business_development_MEAN_low_risk
APPROVED_NAME_GOODS_CATEGORY_Insurance_MEAN_high_risk
APPROVED_NAME_GOODS_CATEGORY_Insurance_MEAN_low_risk
APPROVED_NAME_GOODS_CATEGORY_Weapon_MEAN_high_risk
APPROVED_NAME_GOODS_CATEGORY_Weapon_MEAN_low_risk
REFUSED_prev_missing_MIN_high_risk
REFUSED_prev_missing_MIN_low_risk
REFUSED_prev_missing_MAX_high_risk
REFUSED_prev_missing_MAX_low_risk
CARD_MONTHS_BALANCE_MAX_high_risk
CARD_MONTHS_BALANCE_MAX_low_risk
CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk
CARD_AMT_DRAWINGS_OTHER_

In [39]:
df_test = df_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df_test.head()

,SK_ID_CURR,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
307505,100001,0.717738,-0.717262,-0.665217,-0.575136,-0.322407,-0.047587,-0.465973,-0.213135,-0.148926,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
307506,100005,-1.393267,-0.717262,-0.665217,-0.575136,-0.656056,-0.915649,-0.682529,-0.950708,1.069336,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376
307507,100013,-1.393267,1.394191,-0.665217,-0.575136,0.303184,0.189387,2.874617,0.278581,-0.130371,...,1.665660,-0.162544,1.983688,-0.310707,1.983998,-0.310707,1.983584,-0.310642,-0.329824,2.047603
307508,100028,0.717738,-0.717262,-0.665217,2.201191,1.345836,2.476585,1.465625,2.860089,0.393555,...,1.665660,-0.162544,1.983688,-0.310707,1.983998,-0.310707,1.983584,-0.310642,-0.329824,2.047603
307509,100038,-1.393267,1.394191,1.503268,0.813027,0.094654,0.094652,0.315035,0.266288,-0.782227,...,-0.600363,-0.162544,-0.504112,-0.310707,-0.504033,-0.310707,-0.504138,-0.310642,-0.329824,-0.488376


In [40]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 1870.41 MB
Memory usage of optimization is 2741714580.00 MB
Decreased by -146583525.2%
Memory usage of dataframe is 296.39 MB
Memory usage of optimization is 434406528.00 MB
Decreased by -146563722.3%


,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
0,100002,1.0,-1.393555,-0.717285,-0.665039,-0.575195,0.303223,-0.454590,-0.184937,-0.483643,...,-0.600586,-0.162598,-0.503906,-0.310791,-0.503906,-0.310791,-0.503906,-0.310547,-0.329834,-0.488281
1,100003,0.0,0.717773,-0.717285,1.502930,-0.575195,0.928711,1.770508,0.561523,1.643555,...,-0.600586,-0.162598,-0.503906,-0.310791,-0.503906,-0.310791,-0.503906,-0.310547,-0.329834,-0.488281
2,100004,0.0,-1.393555,1.394531,-0.665039,-0.575195,-0.948242,-1.135742,-1.403320,-1.073242,...,-0.600586,-0.162598,-0.503906,-0.310791,-0.503906,-0.310791,-0.503906,-0.310547,-0.329834,-0.488281
3,100006,0.0,0.717773,-0.717285,-0.665039,-0.575195,-0.322510,-0.689941,0.153442,-0.630859,...,1.666016,-0.162598,-0.503906,3.218750,-0.503906,3.218750,-0.503906,3.218750,-0.329834,2.046875
4,100007,0.0,-1.393555,-0.717285,-0.665039,-0.575195,-0.447510,-0.187622,-0.377441,-0.041046,...,-0.600586,-0.162598,-0.503906,-0.310791,-0.503906,-0.310791,-0.503906,-0.310547,-0.329834,-0.488281


## 学習

### 学習(train)

In [41]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [42]:
baseline = Baseline('SK_ID_CURR')

"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246004, 2225) (61501, 2225)
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
[100]	training's auc: 0.777549	valid_1's auc: 0.755444
[200]	training's auc: 0.793564	valid_1's auc: 0.765622
[300]	training's auc: 0.806894	valid_1's auc: 0.772924
[400]	training's auc: 0.817898	valid_1's auc: 0.778288
[500]	training's auc: 0.827132	valid_1's auc: 0.781952
[600]	training's auc: 0.835387	valid_1's auc: 0.78444
[700]	training's auc: 0.842857	valid_1's auc: 0.786371
[800]	training's auc: 0.849815	valid_1's auc: 0.78775
[900]	training's auc: 0.856392	valid_1's auc: 0.788864
[1000]	training's auc: 0.862606	valid_1's auc: 0.789701
[1100]	training's auc: 0.868506	valid_1's auc: 0.790416
[1200]	training's auc: 0.874047	valid_1's auc: 0.79083
[1300]	training's auc: 0.879268	valid_1's auc: 0.79117
[1400]	training's auc: 0.884152	valid_1's auc: 0.791542
[1500]	training's auc: 0.88872	valid_1's a

In [43]:
train_columns = list(df_train.columns.sort_values(ascending=False))

for col in train_columns:
    print(col)

app_popular_AMT_GOODS_PRICE
app_most_popular_AMT_GOODS_PRICE
app_missing
app_PAYMENT_RATE
app_OWN_CAR_AGE_div_DAYS_EMPLOYED
app_OWN_CAR_AGE_div_DAYS_BIRTH
app_NEW_SCORES_STD
app_NEW_EXT_SOURCES_MEAN
app_INCOME_PER_PERSON
app_INCOME_CREDIT_PERC
app_EXT_SOURCE_prod
app_EXT_SOURCE_3_times_DAYS_EMPLOYED
app_EXT_SOURCE_3_PER_DAYS_BIRTH
app_EXT_SOURCE_2_times_EXT_SOURCE_3
app_EXT_SOURCE_2_times_DAYS_EMPLOYED
app_EXT_SOURCE_2_PER_DAYS_BIRTH
app_EXT_SOURCE_1_times_EXT_SOURCE_3
app_EXT_SOURCE_1_times_EXT_SOURCE_2
app_EXT_SOURCE_1_times_DAYS_EMPLOYED
app_EXT_SOURCE_1_PER_DAYS_BIRTH
app_DAYS_LAST_PHONE_CHANGE_div_DAYS_EMPLOYED
app_DAYS_LAST_PHONE_CHANGE_div_DAYS_BIRTH
app_DAYS_EMPLOYED_div_DAYS_BIRTH
app_DAYS_EMPLOYED_PERC
app_CNT_CHILDREN_div_CNT_FAM_MEMBERS_low_risk
app_CNT_CHILDREN_div_CNT_FAM_MEMBERS_high_risk
app_CNT_CHILDREN_div_CNT_FAM_MEMBERS
app_ANNUITY_INCOME_PERC
app_AMT_INCOME_TOTAL_div_CNT_CHILDREN
app_AMT_INCOME_TOTAL_div_12_AMT_ANNUITY
app_AMT_INCOME_TOTAL_PER_AMT_ANNUITY
app_AMT_I

ACTIVE_AMT_CREDIT_SUM_DEBT_VAR
ACTIVE_AMT_CREDIT_SUM_DEBT_SUM
ACTIVE_AMT_CREDIT_SUM_DEBT_MIN
ACTIVE_AMT_CREDIT_SUM_DEBT_MEAN
ACTIVE_AMT_CREDIT_SUM_DEBT_MAX
ACTIVE_AMT_CREDIT_MAX_OVERDUE_VAR
ACTIVE_AMT_CREDIT_MAX_OVERDUE_SUM
ACTIVE_AMT_CREDIT_MAX_OVERDUE_MIN
ACTIVE_AMT_CREDIT_MAX_OVERDUE_MEAN
ACTIVE_AMT_CREDIT_MAX_OVERDUE_MAX
ACTIVE_AMT_ANNUITY_VAR
ACTIVE_AMT_ANNUITY_SUM
ACTIVE_AMT_ANNUITY_MIN
ACTIVE_AMT_ANNUITY_MEAN
ACTIVE_AMT_ANNUITY_MAX


In [44]:
test_columns = list(df_test.columns.sort_values(ascending=False))

for col in test_columns:
    print(col)

app_popular_AMT_GOODS_PRICE
app_most_popular_AMT_GOODS_PRICE
app_missing
app_PAYMENT_RATE
app_OWN_CAR_AGE_div_DAYS_EMPLOYED
app_OWN_CAR_AGE_div_DAYS_BIRTH
app_NEW_SCORES_STD
app_NEW_EXT_SOURCES_MEAN
app_INCOME_PER_PERSON
app_INCOME_CREDIT_PERC
app_EXT_SOURCE_prod
app_EXT_SOURCE_3_times_DAYS_EMPLOYED
app_EXT_SOURCE_3_PER_DAYS_BIRTH
app_EXT_SOURCE_2_times_EXT_SOURCE_3
app_EXT_SOURCE_2_times_DAYS_EMPLOYED
app_EXT_SOURCE_2_PER_DAYS_BIRTH
app_EXT_SOURCE_1_times_EXT_SOURCE_3
app_EXT_SOURCE_1_times_EXT_SOURCE_2
app_EXT_SOURCE_1_times_DAYS_EMPLOYED
app_EXT_SOURCE_1_PER_DAYS_BIRTH
app_DAYS_LAST_PHONE_CHANGE_div_DAYS_EMPLOYED
app_DAYS_LAST_PHONE_CHANGE_div_DAYS_BIRTH
app_DAYS_EMPLOYED_div_DAYS_BIRTH
app_DAYS_EMPLOYED_PERC
app_CNT_CHILDREN_div_CNT_FAM_MEMBERS_low_risk
app_CNT_CHILDREN_div_CNT_FAM_MEMBERS_high_risk
app_CNT_CHILDREN_div_CNT_FAM_MEMBERS
app_ANNUITY_INCOME_PERC
app_AMT_INCOME_TOTAL_div_CNT_CHILDREN
app_AMT_INCOME_TOTAL_div_12_AMT_ANNUITY
app_AMT_INCOME_TOTAL_PER_AMT_ANNUITY
app_AMT_I

ACTIVE_Month_SIZE_MIN
ACTIVE_Month_SIZE_MEAN
ACTIVE_Month_SIZE_MAX
ACTIVE_Month_MIN_VAR
ACTIVE_Month_MIN_SUM
ACTIVE_Month_MIN_MIN
ACTIVE_Month_MIN_MEAN
ACTIVE_Month_MIN_MAX
ACTIVE_Month_MAX_SUM
ACTIVE_Month_MAX_MIN
ACTIVE_Month_MAX_MEAN
ACTIVE_Month_MAX_MAX
ACTIVE_MONTHS_BALANCE_SIZE_VAR
ACTIVE_MONTHS_BALANCE_SIZE_SUM
ACTIVE_MONTHS_BALANCE_SIZE_MIN
ACTIVE_MONTHS_BALANCE_SIZE_MEAN
ACTIVE_MONTHS_BALANCE_SIZE_MAX
ACTIVE_MONTHS_BALANCE_MIN_VAR
ACTIVE_MONTHS_BALANCE_MIN_SUM
ACTIVE_MONTHS_BALANCE_MIN_MIN
ACTIVE_MONTHS_BALANCE_MIN_MEAN
ACTIVE_MONTHS_BALANCE_MIN_MAX
ACTIVE_MONTHS_BALANCE_MAX_VAR
ACTIVE_MONTHS_BALANCE_MAX_SUM
ACTIVE_MONTHS_BALANCE_MAX_MIN
ACTIVE_MONTHS_BALANCE_MAX_MEAN
ACTIVE_MONTHS_BALANCE_MAX_MAX
ACTIVE_Last_status_X_MEAN_VAR
ACTIVE_Last_status_X_MEAN_SUM_low_risk
ACTIVE_Last_status_X_MEAN_SUM_high_risk
ACTIVE_Last_status_X_MEAN_SUM
ACTIVE_Last_status_X_MEAN_MEAN
ACTIVE_Last_status_X_MEAN_MAX
ACTIVE_Last_status_C_MEAN_VAR
ACTIVE_Last_status_C_MEAN_SUM_low_risk
ACTIVE_Last_sta

In [45]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
2217,app_NEW_EXT_SOURCES_MEAN,323867.949743,2847.840909
2211,app_EXT_SOURCE_2_times_EXT_SOURCE_3,120995.305473,1382.257955
1467,EXT_SOURCE_3,64769.667779,1653.021228
1466,EXT_SOURCE_2,49483.652652,2090.084085
2190,app_AMT_CREDIT_PER_AMT_ANNUITY,33704.206849,2656.269589
2205,app_EXT_SOURCE_1_PER_DAYS_BIRTH,33486.668228,913.662305
2210,app_EXT_SOURCE_2_times_DAYS_EMPLOYED,32250.181018,1841.040812
2191,app_AMT_CREDIT_PER_AMT_GOODS_PRICE,21212.834932,1170.243082
2221,app_PAYMENT_RATE,19752.835089,1274.420932
2189,app_AMT_CREDIT_AMT_GOODS_PRICE,19007.715046,1499.403017


### 学習(test)

In [46]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [47]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [48]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineeringOtherSource1-01.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(97488, 2)


,SK_ID_CURR,TARGET
0,NaN,0.034531
1,NaN,0.112707
2,NaN,0.047484
3,NaN,0.035983
4,NaN,0.136736


## パラメータ・チューニング

In [ ]:
baseline = Baseline('SK_ID_CURR')

sampler = optuna.samplers.TPESampler(seed=123)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective, n_trials=50, n_jobs=5)

[I 2022-06-29 12:07:52,961] A new study created in memory with name: no-name-0341e911-1be4-4300-92a1-f212ff9f5ac7


In [ ]:
trial = study.best_trial
print("auc(best)={:.4f}".format(trial.value))
display(trial.params)

In [ ]:
params_best = trial.params
params_best.update(params_base)
display(params_best)

In [ ]:
train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5
                                    ,params=params_best)

In [ ]:
# 推論用データセットの作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

# predict
test_pred = baseline.predict_lgb(x_test
                       ,id_test
                       ,list_nfold=[0,1,2,3,4])

# make submission-file
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv("./home-credit-default-risk/submit/submission_featureEngineeringOtherSource2-1.csv", index=None)

In [ ]:
del df_train
del df_test
gc.collect()